In [1]:
import pandas as pd

In [2]:
compounding_period={
    "Annually": 1,
    "Semi-Annually": 2,
    "Quarterly": 4, 
    "Monthly": 12
}

payment_periods = {
    "Annually": {"periods_per_year": 1, "months_offset": 12, "day_offset": 0},
    "Semi-Annually": {"periods_per_year": 2, "months_offset": 6, "day_offset": 0},
    "Quarterly": {"periods_per_year": 4, "months_offset": 3, "day_offset": 0},
    "Bi-Monthly": {"periods_per_year": 6, "months_offset": 2, "day_offset": 0},
    "Monthly": {"periods_per_year": 12, "months_offset": 1, "day_offset": 0},
    "Semi-Monthly": {"periods_per_year": 24, "months_offset": 0, "day_offset": 15},
    "Bi-Weekly": {"periods_per_year": 26, "months_offset": 0, "day_offset": 14},
    "Weekly": {"periods_per_year": 52, "months_offset": 0, "day_offset": 7}
}


In [3]:
def amortization(principal, amortization_term, interest, compounding_frequency, payment_frequency, cpr, mortgage_term=0):
    calculated_dict=dict()
    calculated_dict["compounding period"]=compounding_period[compounding_frequency]
    calculated_dict["periods per year"]=payment_periods[payment_frequency]["periods_per_year"]
    calculated_dict["interest rate per payment"] = ((1+(interest/calculated_dict["compounding period"]))**(calculated_dict["compounding period"]/calculated_dict["periods per year"]))-1
    if mortgage_term:
        calculated_dict["renewal period"] = (mortgage_term/12)*calculated_dict["periods per year"]
    else:
        calculated_dict["renewal period"] = 0
    calculated_dict["amortization period"] = (amortization_term/12)*calculated_dict["periods per year"]
    calculated_dict["payment per period"] = ((calculated_dict["interest rate per payment"])*principal)/(1-((1+(calculated_dict["interest rate per payment"]))**(-1*calculated_dict["amortization period"])))
    calculated_dict["smm"] = cpr/calculated_dict["periods per year"]
    calculated_dict["month offset"] = payment_periods[payment_frequency]["months_offset"]
    calculated_dict["day offset"] = payment_periods[payment_frequency]["day_offset"]
    return calculated_dict

In [4]:
# a=amortization(principal=10000, amortization_term=300, mortgage_term=12, interest=5, compounding_frequency="Monthly", payment_frequency="Monthly", cpr=10)

In [5]:
# a

In [6]:
import pandas as pd
from datetime import datetime, timedelta

# Input values
start_date = "13-01-2024"
original_principal = 10000  # Amount
amortization_term_months = 300  # Number of months for the amortization
mortgage_term_months = 12  # Mortgage term (should be same as renewal_period)
interest_rate = 0.05  # 5% annual interest rate
compounding_frequency = "Monthly"  # Compounded monthly
payment_frequency = "Weekly"  # Monthly payments
cpr = 0.1  # 10% constant prepayment rate


a=amortization(principal=original_principal, amortization_term=amortization_term_months, mortgage_term=mortgage_term_months, interest=interest_rate, compounding_frequency=compounding_frequency, payment_frequency=payment_frequency, cpr=cpr)


compounding_period = a["compounding period"]  # Compounding frequency per year
periods_per_year = a["periods per year"]  # 12 periods per year
interest_rate_perpayment = a["interest rate per payment"]  # 0.417% interest per period
renewal_period = a["renewal period"]  # Annual renewal
amortization_period = a["amortization period"]  # 300 months
payment_per_period = a["payment per period"]  # Fixed payment amount
smm = a["smm"]  # 0.83% SMM (Single Monthly Mortality rate)
month_offset = a["month offset"]  # Payment month offset
day_offset = a["day offset"]  # Day offset

# Convert the start date to a datetime object
start_date = datetime.strptime(start_date, "%d-%m-%Y")

# Initialize an empty list to store the rows for the DataFrame
amortization_schedule = []

# Set the initial values for period 0
period = 0
date = start_date
opening_balance = original_principal
new_origination = original_principal
closing_balance = new_origination

# Loop over the periods
for period in range(int(a["amortization period"]) + 1):  # +1 to include period 0
    # Calculate the interest for the period
    interest = (opening_balance * interest_rate_perpayment)

    # Payment is the minimum of payment_per_period and the remaining balance (including interest)
    if opening_balance > payment_per_period:
        payment = (payment_per_period)
    else:
        payment = (opening_balance + interest)
    
    # Calculate principal paid for the period
    principal = payment - interest
    
    # Calculate prepayment
    if opening_balance - principal > 0:
        prepayment = (opening_balance * smm)
    else:
        prepayment = 0
    
    if not(period):
        opening_balance=0
        payment=0
        interest=0
        principal=0
        prepayment=0

    else:
        new_origination=0
        

    maturity=0
    # Apply prepayment, maturity, and calculate closing balance
    if period < renewal_period or mortgage_term_months==0:
        # For the first 'mortgage_term_months', no maturity (i.e., closing_balance is just reduced by prepayment and principal)
        closing_balance = opening_balance - prepayment - principal + new_origination
    elif period >= renewal_period or period>=amortization_period:
        # After mortgage term ends, apply maturity
        maturity = opening_balance - principal - prepayment # Remaining balance after regular payments
        closing_balance = 0  # After maturity, the loan is fully paid off
        
    
    # Prepare the data for this period
    row = {
        "period": period,
        "date": date.strftime("%d-%m-%Y"),
        "opening_balance": opening_balance,
        "payment": payment,
        "interest": interest,
        "principal": principal,
        "prepayment": prepayment,
        "new_origination": new_origination,
        "maturity": maturity if period >= mortgage_term_months else 0,
        "closing_balance": closing_balance
    }
    
    # Add the row to the amortization schedule
    amortization_schedule.append(row)
    
    # Update the opening balance for the next period
    opening_balance = closing_balance  # Opening balance of the next period is the closing balance of the current period
    
    # Increment the date
    if payment_frequency=="Weekly":
        date = date + timedelta(days=7)
    elif payment_frequency=="Bi-Weekly":
        date = date + timedelta(days=15)
    elif payment_frequency=="Semi-Monthly":
        date = date + timedelta(days=15)
    elif payment_frequency=="Monthly":
        date = date + timedelta(days=30)  # Adding 30 days to get the next month's date
    if (period >= renewal_period or period>=amortization_period) and mortgage_term_months!=0:
        break

# Create the DataFrame from the amortization schedule list
df_amortization = pd.DataFrame(amortization_schedule)

# Display the resulting DataFrame
print(df_amortization)


    period        date  opening_balance    payment  interest  principal  \
0        0  13-01-2024         0.000000   0.000000  0.000000   0.000000   
1        1  20-01-2024     10000.000000  13.468973  9.600013   3.868960   
2        2  27-01-2024      9976.900271  13.468973  9.577837   3.891136   
3        3  03-02-2024      9953.822789  13.468973  9.555683   3.913290   
4        4  10-02-2024      9930.767532  13.468973  9.533550   3.935423   
5        5  17-02-2024      9907.734479  13.468973  9.511438   3.957535   
6        6  24-02-2024      9884.723608  13.468973  9.489348   3.979625   
7        7  02-03-2024      9861.734899  13.468973  9.467278   4.001695   
8        8  09-03-2024      9838.768330  13.468973  9.445230   4.023742   
9        9  16-03-2024      9815.823879  13.468973  9.423204   4.045769   
10      10  23-03-2024      9792.901526  13.468973  9.401198   4.067775   
11      11  30-03-2024      9770.001248  13.468973  9.379214   4.089759   
12      12  06-04-2024   

In [7]:
a

{'compounding period': 12,
 'periods per year': 52,
 'interest rate per payment': 0.0009600013100943272,
 'renewal period': 52.0,
 'amortization period': 1300.0,
 'payment per period': 13.46897293717892,
 'smm': 0.0019230769230769232,
 'month offset': 0,
 'day offset': 7}

In [8]:
pd.DataFrame(list(a.items()), columns=['Key', 'Value'])

,Key,Value
0,compounding period,12.000000
1,periods per year,52.000000
2,interest rate per payment,0.000960
3,renewal period,52.000000
4,amortization period,1300.000000
5,payment per period,13.468973
6,smm,0.001923
7,month offset,0.000000
8,day offset,7.000000


In [9]:
df_amortization

,period,date,opening_balance,payment,interest,principal,prepayment,new_origination,maturity,closing_balance
0,0,13-01-2024,0.000000,0.000000,0.000000,0.000000,0.000000,10000,0.000000,10000.000000
1,1,20-01-2024,10000.000000,13.468973,9.600013,3.868960,19.230769,0,0.000000,9976.900271
2,2,27-01-2024,9976.900271,13.468973,9.577837,3.891136,19.186347,0,0.000000,9953.822789
3,3,03-02-2024,9953.822789,13.468973,9.555683,3.913290,19.141967,0,0.000000,9930.767532
4,4,10-02-2024,9930.767532,13.468973,9.533550,3.935423,19.097630,0,0.000000,9907.734479
5,5,17-02-2024,9907.734479,13.468973,9.511438,3.957535,19.053336,0,0.000000,9884.723608
6,6,24-02-2024,9884.723608,13.468973,9.489348,3.979625,19.009084,0,0.000000,9861.734899
7,7,02-03-2024,9861.734899,13.468973,9.467278,4.001695,18.964875,0,0.000000,9838.768330
8,8,09-03-2024,9838.768330,13.468973,9.445230,4.023742,18.920708,0,0.000000,9815.823879
9,9,16-03-2024,9815.823879,13.468973,9.423204,4.045769,18.876584,0,0.000000,9792.901526


In [10]:
df_amortization.round(2).to_excel("forst.xlsx")